## packages

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from collections import Counter
import math

## data


In [4]:
dict={'M':1,'S':2,'L':3}
def creat_data():
    df=pd.read_csv('./c3_data.csv')
    data=np.array(df)
    X,y=data[:,:-1],data[:,-1]
    # print(X,y)
    for x in X:
        x[1]=dict[x[1]]
    return  X.astype(np.int64),y.astype(np.int64)

In [5]:
X,y=creat_data()
X,y
# print(X,y)

(array([[1, 1],
        [1, 1],
        [1, 2],
        [1, 2],
        [2, 2],
        [2, 1],
        [2, 1],
        [2, 3],
        [2, 3],
        [3, 3],
        [3, 1],
        [3, 1],
        [3, 3],
        [3, 3]], dtype=int64),
 array([-1,  1,  1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1, -1],
       dtype=int64))

## sklearn

In [6]:
X,y=creat_data()
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train,y_train=X,y

In [8]:
X_train[0], y_train[0]

(array([1, 1], dtype=int64), -1)

In [9]:
from sklearn.naive_bayes import GaussianNB

In [10]:
clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB()

In [11]:
clf.predict([[2,dict['S']]])

array([1], dtype=int64)

## Naive

In [13]:
X_train,y_train=creat_data()

In [14]:
# seperate_by_class():
seperated_date={}
for value,label in zip(X_train.tolist(),y_train.tolist()):
    if label not in seperated_date:
        seperated_date[label]=[]
    seperated_date[label].append(value)
print(seperated_date)

{-1: [[1, 1], [1, 2], [2, 2], [2, 1], [3, 3]], 1: [[1, 1], [1, 2], [2, 1], [2, 3], [2, 3], [3, 3], [3, 1], [3, 1], [3, 3]]}


In [16]:
#p_label_calucate()
p_label={}
for i in seperated_date:
    # print(len(seperated_date[i]))
    p_label[i]=len(seperated_date[i])/len(X_train)
print(p_label)

{-1: 0.35714285714285715, 1: 0.6428571428571429}


In [13]:
## 代码可读性非常的重要。直接决定了寿命。如何写出呢？多写多读。写了让别人读，读别人的好的地方要借鉴。
# 不要为了酷炫牺牲可读性。不要投机取巧。
# 必须要做模块化，非常重要，人脑是技巧。
## 模块之前不要实现循环依赖。形成偏序集。


In [18]:
# summary_label:calculate the condition probility
def summary_label(value):
    a=[]
    for column in zip(*value):
        column=list(column)
        p_value={}
        for i in set(column):
            p_value[i]=column.count(i)/len(value)
        a.append(p_value.copy())
    return a.copy()

In [20]:
## calculate_p_value
p_value={label:summary_label(value) for label,value in seperated_date.items()}
p_value
    

{-1: [{1: 0.4, 2: 0.4, 3: 0.2}, {1: 0.4, 2: 0.4, 3: 0.2}],
 1: [{1: 0.2222222222222222, 2: 0.3333333333333333, 3: 0.4444444444444444},
  {1: 0.4444444444444444, 2: 0.1111111111111111, 3: 0.4444444444444444}]}

### predict

In [22]:
x=[3,dict['S']]
max_label=0
max_p=0
for label in p_label:
    p=p_label[label]
    for i in range(len(x)):
        p=p*p_value[label][i][x[i]]
    if max_p<p:
        max_p=p
        max_label=label
max_label

1

relfection:
* .this algorithm must make sure that every kind of instance occurs in the data of training

## 使用class进行存储

In [32]:
X_train,y_train=creat_data()

In [47]:
class model:
    def __init__(self):
        self.p_label={}
        self.p_value={}
    def seperate_by_class(self,X_train,y_train):
        seperated_date={}
        for value,label in zip(X_train.tolist(),y_train.tolist()):
            if label not in seperated_date:
                seperated_date[label]=[]
            seperated_date[label].append(value)
        return seperated_date
    def p_label_calucate(self,seperated_date):
        p_label={}
        for i in seperated_date:
            p_label[i]=len(seperated_date[i])/len(X_train)
        return p_label

        # summary_label:calculate the condition probility
    def summary_label(self,value):
        a=[]
        for column in zip(*value):
            column=list(column)
            p_value={}
            for i in set(column):
                p_value[i]=column.count(i)/len(value)
            a.append(p_value.copy())
        return a.copy()

    def calculate_p_value(self,seperated_date):
        p_value={label:self.summary_label(value) for label,value in seperated_date.items()}
        return p_value

    def train(self,X_train,y_train):
        seperated_date=self.seperate_by_class(X_train,y_train)
        self.p_label=self.p_label_calucate(seperated_date)
        self.p_value=self.calculate_p_value(seperated_date)
    
    def predict(self,x):
        p_label=self.p_label
        p_value=self.p_value
        max_label=0
        max_p=0
        for label in p_label:
            p=p_label[label]
            for i in range(len(x)):
                p=p*p_value[label][i][x[i]]
            if max_p<p:
                max_p=p
                max_label=label
        print(max_label)
    

In [48]:
naiveByes=model()

In [49]:
naiveByes.train(X_train,y_train)

In [50]:
naiveByes.predict([3,dict['S']])

1


## 贝叶斯估计
加上拉普拉斯平滑（Laplacian smoothing)

In [54]:
class model_lam:
    def __init__(self,lam=1): # lambda >0
        self.p_label={}
        self.p_value={}
        self.lam=lam

    def seperate_by_class(self,X_train,y_train):
        seperated_date={}
        for value,label in zip(X_train.tolist(),y_train.tolist()):
            if label not in seperated_date:
                seperated_date[label]=[]
            seperated_date[label].append(value)
        return seperated_date
    def p_label_calucate(self,seperated_date,X_train):
        p_label={}
        for i in seperated_date:
            p_label[i]=len(seperated_date[i]+self.lam)/(len(X_train)+len(seperated_date)*self.lam)  ## add the lambda  
        return p_label

        # summary_label:calculate the condition probility
    def summary_label(self,value):
        a=[]
        for column in zip(*value):
            column=list(column)
            p_value={}
            a=set(column)
            Sj=len(a)
            for i in a:
                p_value[i]=(self.lam+column.count(i))/(len(value)+self.lam+Sj)  ## add the lambda 
            a.append(p_value.copy())
        return a.copy()

    def calculate_p_value(self,seperated_date,X_train):
        p_value={label:self.summary_label(value) for label,value in seperated_date.items()}
        return p_value

    def train(self,X_train,y_train):
        seperated_date=self.seperate_by_class(X_train,y_train)
        self.p_label=self.p_label_calucate(seperated_date,X_train)
        self.p_value=self.calculate_p_value(seperated_date)
    
    def predict(self,x):
        p_label=self.p_label
        p_value=self.p_value
        max_label=0
        max_p=0
        for label in p_label:
            p=p_label[label]
            for i in range(len(x)):
                p=p*p_value[label][i][x[i]]
            if max_p<p:
                max_p=p
                max_label=label
        print(max_label)
    

In [55]:
Bayes=model_lam()

In [56]:
naiveByes.train(X_train,y_train)

In [57]:
naiveByes.predict([3,dict['S']])

1
